In [111]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn import model_selection
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Akshata
[nltk_data]     Pandit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [112]:
data=pd.read_csv("C:\\Users\\Akshata Pandit\\OneDrive\\vscode1\\IFND.csv",encoding="latin-1")

In [113]:
print(data)

          id                                          Statement  \
0          2  WHO praises India's Aarogya Setu app, says it ...   
1          3  In Delhi, Deputy US Secretary of State Stephen...   
2          4  LAC tensions: China's strategy behind delibera...   
3          5  India has signed 250 documents on Space cooper...   
4          6  Tamil Nadu chief minister's mother passes away...   
...      ...                                                ...   
56709  56711  Fact Check: This is not Bruce Lee playing ping...   
56710  56712  Fact Check: Did Japan construct this bridge in...   
56711  56713  Fact Check: Viral video of Mexico earthquake i...   
56712  56714  Fact Check: Ballet performance by Chinese coup...   
56713  56715  Fact Check: Is this little boy crossing into J...   

                                                   Image         Web  \
0      https://cdn.dnaindia.com/sites/default/files/s...    DNAINDIA   
1      https://cdn.dnaindia.com/sites/default/files

In [114]:
data.isnull().sum()

id               0
Statement        0
Image            0
Web              0
Category         0
Date         11321
Label            0
dtype: int64

In [115]:
data.head()

,id,Statement,Image,Web,Category,Date,Label
0,2,"WHO praises India's Aarogya Setu app, says it ...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
1,3,"In Delhi, Deputy US Secretary of State Stephen...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,VIOLENCE,Oct-20,TRUE
2,4,LAC tensions: China's strategy behind delibera...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,TERROR,Oct-20,TRUE
3,5,India has signed 250 documents on Space cooper...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
4,6,Tamil Nadu chief minister's mother passes away...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,ELECTION,Oct-20,TRUE


In [116]:
data.shape

(56714, 7)

In [117]:
data.fillna('')

,id,Statement,Image,Web,Category,Date,Label
0,2,"WHO praises India's Aarogya Setu app, says it ...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
1,3,"In Delhi, Deputy US Secretary of State Stephen...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,VIOLENCE,Oct-20,TRUE
2,4,LAC tensions: China's strategy behind delibera...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,TERROR,Oct-20,TRUE
3,5,India has signed 250 documents on Space cooper...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
4,6,Tamil Nadu chief minister's mother passes away...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,ELECTION,Oct-20,TRUE
...,...,...,...,...,...,...,...
56709,56711,Fact Check: This is not Bruce Lee playing ping...,https://akm-img-a-in.tosshub.com/indiatoday/im...,INDIATODAY,MISLEADING,11-2019,Fake
56710,56712,Fact Check: Did Japan construct this bridge in...,https://akm-img-a-in.tosshub.com/indiatoday/im...,INDIATODAY,COVID-19,10-2019,Fake
56711,56713,Fact Check: Viral video of Mexico earthquake i...,https://akm-img-a-in.tosshub.com/indiatoday/im...,INDIATODAY,MISLEADING,10-2019,Fake
56712,56714,Fact Check: Ballet performance by Chinese coup...,https://akm-img-a-in.tosshub.com/indiatoday/im...,INDIATODAY,COVID-19,9-2019,Fake


In [118]:
data.isnull().sum()

id               0
Statement        0
Image            0
Web              0
Category         0
Date         11321
Label            0
dtype: int64

In [119]:
data["content"]=data["Category"]+'   '+data["Statement"]

In [120]:
data=data[['Label','content']]

In [121]:
data

,Label,content
0,TRUE,COVID-19 WHO praises India's Aarogya Setu ap...
1,TRUE,"VIOLENCE In Delhi, Deputy US Secretary of St..."
2,TRUE,TERROR LAC tensions: China's strategy behind...
3,TRUE,COVID-19 India has signed 250 documents on S...
4,TRUE,ELECTION Tamil Nadu chief minister's mother ...
...,...,...
56709,Fake,MISLEADING Fact Check: This is not Bruce Lee...
56710,Fake,COVID-19 Fact Check: Did Japan construct thi...
56711,Fake,MISLEADING Fact Check: Viral video of Mexico...
56712,Fake,COVID-19 Fact Check: Ballet performance by C...


In [122]:
data["content"][28]

'GOVERNMENT   Seven Indians kidnapped in Libya in September released'

In [123]:
# stemming
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [124]:
data['content']=data['content'].apply(stemming)

In [125]:
X_train,X_test,Y_train,Y_test=train_test_split(data['content'],data['Label'])

In [126]:
X_train

33459    violenc patna high court seek repli govt poor ...
34565    govern sai baba birthplac row shop eateri loca...
43088            govern pictur man pee pm modi poster fake
10557     govern bodi year old man throat slit found banda
50165    mislead waqf board issu denial claim babri hos...
                               ...                        
31871    govern csr disclosur rule set tighten increas ...
19016    violenc hc quash charg sheet supriyo outrag mo...
22889        terror jharkhand poll candid age increas year
27619    govern enough talk aap delhi sop bjp plan show...
53318    govern believ fake twitter handl attribut isro...
Name: content, Length: 42535, dtype: object

In [127]:
# TF-IDF vectorization
tfidf_vectorizer=TfidfVectorizer()
X_train_tfidf=tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf=tfidf_vectorizer.transform(X_test)

In [128]:
rf=RandomForestClassifier(random_state=42)
rf.fit(X_train_tfidf,Y_train)

RandomForestClassifier(random_state=42)

In [129]:
p_pred=rf.predict(X_test_tfidf)

In [130]:
conf_matrix = confusion_matrix(Y_test, p_pred)
class_report = classification_report(Y_test, p_pred)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

Confusion Matrix:
[[4224  426]
 [ 130 9399]]

Classification Report:
              precision    recall  f1-score   support

        Fake       0.97      0.91      0.94      4650
        TRUE       0.96      0.99      0.97      9529

    accuracy                           0.96     14179
   macro avg       0.96      0.95      0.95     14179
weighted avg       0.96      0.96      0.96     14179



In [131]:
def predict(text):
    stemmed=stemming(text)
    texxt=tfidf_vectorizer.transform([stemmed])
    prediction=rf.predict(texxt)
    if prediction[0]=='Fake':
        label="news is fake"
    else:
        label="news is real"
    return label
input="COVID-19 Fact Check: Did Japan construct this bridge in 24 hours?" 
predict(input)

'news is fake'

In [132]:
#svm classifier


In [133]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train_tfidf, Y_train)

SVC(kernel='linear', random_state=42)

In [134]:
y_pred_svm = svm_classifier.predict(X_test_tfidf)

In [135]:
print("train accurracy :",accuracy_score(y_pred_svm,Y_test))

train accurracy : 0.9569081035333945


In [136]:
conf_matrix_svm = confusion_matrix(Y_test, y_pred_svm)
class_report_svm = classification_report(Y_test, y_pred_svm)

In [137]:
print("SVM - Confusion Matrix:")
print(conf_matrix_svm)
print("\nSVM - Classification Report:")
print(class_report_svm)

SVM - Confusion Matrix:
[[4156  494]
 [ 117 9412]]

SVM - Classification Report:
              precision    recall  f1-score   support

        Fake       0.97      0.89      0.93      4650
        TRUE       0.95      0.99      0.97      9529

    accuracy                           0.96     14179
   macro avg       0.96      0.94      0.95     14179
weighted avg       0.96      0.96      0.96     14179



In [138]:
def predict(text):
    stemmed=stemming(text)
    texxt=tfidf_vectorizer.transform([stemmed])
    prediction=rf.predict(texxt)
    if prediction[0]=='Fake':
        label="news is fake"
    else:
        label="news is real"
    return label
input="COVID-19 Fact Check: Did Japan construct this bridge in 24 hours?" 
predict(input)

'news is fake'

In [139]:
#logisticregression

In [140]:
from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression(random_state=42)
lr_classifier.fit(X_train_tfidf, Y_train)

LogisticRegression(random_state=42)

In [141]:
y_pred_lr = lr_classifier.predict(X_test_tfidf)

In [142]:
print("train accurracy :",accuracy_score(y_pred_lr,Y_test))

train accurracy : 0.9485154101135482


In [143]:
conf_matrix_lr = confusion_matrix(Y_test, y_pred_lr)
class_report_lr = classification_report(Y_test, y_pred_lr)

In [144]:
print("Logistic Regression - Confusion Matrix:")
print(conf_matrix_lr)
print("\nLogistic Regression - Classification Report:")
print(class_report_lr)

Logistic Regression - Confusion Matrix:
[[4022  628]
 [ 102 9427]]

Logistic Regression - Classification Report:
              precision    recall  f1-score   support

        Fake       0.98      0.86      0.92      4650
        TRUE       0.94      0.99      0.96      9529

    accuracy                           0.95     14179
   macro avg       0.96      0.93      0.94     14179
weighted avg       0.95      0.95      0.95     14179



In [145]:
def predict(text):
    stemmed=stemming(text)
    texxt=tfidf_vectorizer.transform([stemmed])
    prediction=rf.predict(texxt)
    if prediction[0]=='Fake':
        label="news is fake"
    else:
        label="news is real"
    return label
input="COVID-19 Fact Check: Did Japan construct this bridge in 24 hours?" 
predict(input)

'news is fake'